In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os


2021-12-15 05:40:24.741740: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-15 05:40:24.741768: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
file = open("shakespeare.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

data = data.split()
data = ' '.join(data)
data[:500]
len(data)




113892

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]


[1, 41, 35, 266, 4, 479, 411, 28, 594, 595, 23, 266, 14, 13, 1]

In [4]:
len(sequence_data)


20984

In [5]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)



3856


In [6]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]


The Length of sequences are:  20981


array([[  1,  41,  35, 266],
       [ 41,  35, 266,   4],
       [ 35, 266,   4, 479],
       [266,   4, 479, 411],
       [  4, 479, 411,  28],
       [479, 411,  28, 594],
       [411,  28, 594, 595],
       [ 28, 594, 595,  23],
       [594, 595,  23, 266],
       [595,  23, 266,  14]])

In [7]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

print("Data: ", X[:10])
print("Response: ", y[:10])


Data:  [[  1  41  35]
 [ 41  35 266]
 [ 35 266   4]
 [266   4 479]
 [  4 479 411]
 [479 411  28]
 [411  28 594]
 [ 28 594 595]
 [594 595  23]
 [595  23 266]]
Response:  [266   4 479 411  28 594 595  23 266  14]


In [8]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))


2021-12-15 05:40:27.054987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-15 05:40:27.055013: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-15 05:40:27.055028: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG157): /proc/driver/nvidia/version does not exist
2021-12-15 05:40:27.055220: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             38560     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 3856)              3859856   
                                                                 
Total params: 16,947,416
Trainable params: 16,947,416
Non-trainable params: 0
_________________________________________________________________


In [11]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)


('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=10, batch_size=64, callbacks=[checkpoint])


Epoch 1/10
328/328 [==============================] - ETA: 0s - loss: 6.9221
Epoch 00001: loss improved from inf to 6.92209, saving model to next_words.h5
328/328 [==============================] - 53s 155ms/step - loss: 6.9221
Epoch 2/10
328/328 [==============================] - ETA: 0s - loss: 6.5608
Epoch 00002: loss improved from 6.92209 to 6.56083, saving model to next_words.h5
328/328 [==============================] - 51s 155ms/step - loss: 6.5608
Epoch 3/10
328/328 [==============================] - ETA: 0s - loss: 6.3461
Epoch 00003: loss improved from 6.56083 to 6.34614, saving model to next_words.h5
328/328 [==============================] - 50s 154ms/step - loss: 6.3461
Epoch 4/10
328/328 [==============================] - ETA: 0s - loss: 6.0941
Epoch 00004: loss improved from 6.34614 to 6.09415, saving model to next_words.h5
328/328 [==============================] - 51s 154ms/step - loss: 6.0941
Epoch 5/10
328/328 [==============================] - ETA: 0s - loss: 5.8459

In [13]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word


In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue


Enter your line: This eBook is for the use of anyone anywhere at no cost and with
['cost', 'and', 'with']
almost
